# Qlunc's presentation

Qlunc stands for **Q**uantification of **l**idar **unc**ertainty. It is stored as a public SWE [repository](https://github.com/SWE-UniStuttgart/Qlunc)

Main Qlunc's structure:
![Qlunc basic structure image](Tutorial_Images/Qlunc_GralStructure.JPG)

## Installation

1. Install [anaconda](https://anaconda.org/anaconda/conda)

2. Download or [clone](https://github.com/SWE-UniStuttgart/Qlunc.git) Qlunc´s repository

3. Create an environment and install dependencies. Dependencies list is in the repository. Environment yaml.

Anaconda prompt:

> conda env create -f environment.yml

> conda activate <environment name> (by default Qlunc_Env)
    

    

## Creating objects as python classes (*Qlunc_Classes.py*)
The goal is to create a lidar digital twin to assess the uncertainty of an actual one, before it is built. First we create the lidar components by using python classes. We give some parameter values to these classes, characterizing the components, inculding a function (`uncertainty`), which computes the uncertainty of each "object" . For instance, you might want to create three components with some characteristics defining them and a fucntion describing each one's uncertainty:

In [ ]:
# Component 1:
class component1():
    def __init__(self, name, param1, param2, comp1_uncertainty):
        self.name              = name
        self.param1            = param1
        self.param2            = param2
        self.uncertainty       = comp1_uncertainty # python module computing uncertainty of componentA. Defined by the user.

# Component 2:
class component2():
    def __init__(self, name, param1, param2, comp2_uncertainty):
        self.name              = name
        self.param1            = param1
        self.param2            = param2
        self.uncertainty       = comp2_uncertainty # python module computing uncertainty of componentB. Defined by the user.

# Component 3:
class component3():
    def __init__(self, name, param1, param2,param3 ,comp3_uncertainty):
        self.name              = name
        self.param1            = param1
        self.param2            = param2
        self.param3            = param3
        self.uncertainty       = comp3_uncertainty # python module computing uncertainty of componentC. Defined by the user.

Then, this "virtual objects", representing the actual lidar components, are encapsulated into another python class representing the actual lidar module. So, having the components we can create 2 different modules; `module1` and `module2`, each of them containing two and one of the components created above, respectively. They also contain a uncertainty function which computes uncertainty expansion among the lidar components we include in the module.


In [ ]:
# Module 1:
class module1():
    def __init__(self,name,component1,component2,mod1_uncertainty):
        self.name             = name
        self.component1       = Component1 # here should have the name of the component instance
        self.component2       = Component2 # here should have the name of the component instance
        self.uncertainty      = mod1_uncertainty # python module computing uncertainty of module1. Defined by the user.

# Module 2:
class module2():
    def __init__(self,name,component1,mod2_uncertainty):
        self.name             = name
        self.component1       = Component3 # here should have the name of the component instance
        self.uncertainty      = mod2_uncertainty # python module computing uncertainty of module2. Defined by the user. GUM

Once the modules are built up, we can encapsulate them into the lidar class, representing the lidar device:

In [ ]:
# Lidar object:
class lidar():
    def __init__(self, name,module1, module2, lidar_uncertainty):
        self.name        = name
        self.module1     = module1 # here should have the name of the module instance
        self.module2     = module2 # here should have the name of the module instance
        self.uncertainty = lidar_uncertainty # python module computing uncertainty of lidar device. Defined by the user. GUM

## **Instantianting process (*Qlunc_Instantiate.py*)
We've created so far an empty structure of a lidar. Now we fill in the components, modules and the lidar itself by giving values to the variables and instantiating the corresponding classes.

All the parameters values are inputted through a yaml file (*Qlunc_inputs.yml*). We do this manually here to show the process:
### Components

In [ ]:
Component1 = component1(name              = 'Comp1',
                        param1            = 'parameter1_Comp1',
                        param2            = 'parameter2_Comp1',
                        comp1_uncertainty = 'unc_func_Comp1')

Component2 = component2(name              = 'Comp2',
                        param1            = 'parameter1_Comp2',
                        param2            = 'parameter2_Comp2',
                        comp2_uncertainty = 'unc_func_Comp2')

Component3 = component3(name              = 'Comp3',
                        param1            = 'parameter1_Comp3',
                        param2            = 'parameter2_Comp3',
                        param3            = 'parameter3_Comp3',
                        comp3_uncertainty = 'unc_func_Comp3')

In [ ]:
print(Component1.name)
print(Component1.param1)
print(Component2.param2)
print(Component3.uncertainty)


### Modules

In [ ]:
Module1 = module1(name             = 'Mod1',
                  component1       = Component1,
                  component2       = Component2,
                  mod1_uncertainty ='unc_func_Mod1')
Module2 = module2(name             = 'Mod2',
                component1         = Component3,
                mod2_uncertainty   = 'unc_func_Mod2')

In [ ]:
print('fragment'+str(Module1.component1.param1))
print(Module2.component1.param3)

### Lidar

In [ ]:
Lidar= lidar(name='Caixa1',module1=Module1,module2=Module2,lidar_uncertainty='Lidar_unc_func')

In [ ]:
print(Lidar.uncertainty)

## **How do we compute uncertainties**
For each lidar component, a research through literature has provide the methods leading to noise in the signal added by the different processes in the photodetector. Resources about applied methods can be found in the metadata. 
The model applies a MC to calculate the error in pointing accuracy due to uncertatinty in the parameters of the scanning system. These uncertainties are provided by the manufacture in terms of standars deviations.

![Uncertainty chain](Tutorial_Images/FlowChartUnc.JPG)

Qlunc contains a python module to specifically import the needed packages and python libraries

In [ ]:
import os
# Importmodules
os.chdir('../Utils')
from Qlunc_ImportModules import *

### **Scanner  (VAD)**
The method to calculate the distance error in pointing accuracy is based on a MonteCaro simulation. The method applies a zero mean gaussian noise over each point in the pattern, with stdv specified by the user (`stdv_focus_dist`,`stdv_cone_angle` and `stdv_azimuth`). The method also accounts for inclinometers error.

#### Scanner parameters

In [ ]:
Lidar_origin=[0,0,0]
sample_rate_count=0
azimuth=np.linspace(1,360,15)
focus_dist=[40]*len(azimuth)
cone_angle=[20]*len(azimuth)

#### Standard deviation values

In [ ]:
stdv_azimuth    = 0.4
stdv_cone_angle = 0.5
stdv_focus_dist = 0.8

#### Initial definitions

In [ ]:
Distance      = []
X             = []
Y             = []
Z             = []
X0            = []
Y0            = []
Z0            = []
NoisyX        = []
NoisyY        = []
NoisyZ        = []
Mean_Distance = []
Stdv_Distance = []

#### Adding noise to the theoretical position:
To perform the MonteCarlo simulation the method uses 10.000 samples. For each point in the pattern 10.000 combinations are computed and stored in the variable `Distance`

In [ ]:
n=10000 # Number of cases to combine
counter=0 # Counter --> for each point in the pattern
for focus_dist_or,cone_angle_or,azimuth_or in zip(focus_dist,cone_angle,azimuth):
    
    # Create the noise based on the standard deviation of the pointing accuracy parameters of the scanner.
    del_focus_dist = np.array(np.random.normal(0,stdv_focus_dist,n)) 
    del_cone_angle = np.array(np.random.normal(0,stdv_cone_angle,n))
    del_azimuth= np.array(np.random.normal(0,stdv_azimuth,n))
    
    # Add noise to te coordinates separately
    noisy_focus_dist = focus_dist_or + del_focus_dist
    noisy_cone_angle = cone_angle_or + del_cone_angle 
    noisy_azimuth    = azimuth_or    + del_azimuth
    
    # Transform to euclidean.
    
    # Euclidean coordinates of the theoretical measuring points
    x0 = (focus_dist_or)*np.cos(np.deg2rad(azimuth_or))*np.sin(np.deg2rad(cone_angle_or)) + Lidar_origin[0]
    y0 = (focus_dist_or)*np.sin(np.deg2rad(azimuth_or))*np.sin(np.deg2rad(cone_angle_or)) + Lidar_origin[1]
    z0 = (focus_dist_or)*np.cos(np.deg2rad(cone_angle_or))+ Lidar_origin[2] + sample_rate_count

    
    # Euclidean coordinates of the noisy points:            
    x = noisy_focus_dist*np.cos(np.deg2rad(noisy_azimuth))*np.sin(np.deg2rad(noisy_cone_angle))
    y = noisy_focus_dist*np.sin(np.deg2rad(noisy_azimuth))*np.sin(np.deg2rad(noisy_cone_angle) )
    z = noisy_focus_dist*np.cos(np.deg2rad(noisy_cone_angle))+ sample_rate_count

    # Distance between theoretical and actual mesuring points.
    Distance.append(np.sqrt((x-x0)**2+(y-y0)**2)+(z-z0)**2)
    Mean_Distance.append(np.mean(Distance))
    Stdv_Distance.append(np.std(Distance))
    
    # Storing coordinates:
    X0.append(x0)
    Y0.append(y0)
    Z0.append(z0)
    X.append(x)
    Y.append(y)
    Z.append(z)
    NoisyX.append(X[counter][0])
    NoisyY.append(Y[counter][0])
    NoisyZ.append(Z[counter][0])
    counter+=1

Noisy_Coord=[NoisyX,NoisyY,NoisyZ]
Coord=[X0,Y0,Z0]
len(Mean_Distance)

In [ ]:
print('Mean Distance[m] = ' + str(Mean_Distance))

In [ ]:
print('STDV Distance[m] = ' +str(Stdv_Distance))

In [ ]:
## Ploting:
fig,axs4 = plt.subplots(figsize=(8,8))  
axs4=plt.axes(projection='3d')
axs4.plot(Lidar_origin[0],Lidar_origin[1],Lidar_origin[2],'go',markersize=15,label='{} coordinates [{},{},{}]'.format('Lidar',Lidar_origin[0],Lidar_origin[1],Lidar_origin[2]))
axs4.plot(X0,Y0,Z0,'bo',label='Theoretical measuring point')
axs4.plot(NoisyX,NoisyY,NoisyZ,'ro',label='Actual measuring point')
axs4.set_xlim3d(-35,35)
axs4.set_ylim3d(-35,35)
axs4.set_zlim3d(-5,45)

axs4.set_xlabel('x [m]',fontsize=34)#,orientation=plot_param['tick_labelrotation'])
axs4.set_ylabel('y [m]',fontsize=34)#,orientation=plot_param['tick_labelrotation'])
axs4.set_zlabel('z [m]',fontsize=34)        
axs4.set_title('Lidar pointing accuracy (VAD)',fontsize=45)
axs4.legend()

In [ ]:
print(X0)
print(len(X0))
print(X)
print(len(X[0]))

### **Photodetector**

#### Input parameters

In [ ]:
# Physical constants:
k = 1.38064852e-23 # Boltzman constant:[m^2 kg s^-2 K^-1]
h = 6.6207004e-34 # Plank constant [m^2 kg s^-1]
e = 1.60217662e-19 # electron charge [C]
c = 2.99792e8 #speed of light [m s^-1]
# Atmospheric temperature
Temperature=300

# Photodetector parameters
name             = 'Photodetector1'  # Introduce your photodetector name.
Wavelength       = 1550.e-9          # Lidar woking wavelength
Photo_BandWidth  = 380.e+9           # Photodetector bandwidth
Load_Resistor    = 50                # [ohms]
Photo_efficiency = .85               # Photodetector efficiency [-]
Dark_Current     = 5e-9              # [A]. Dark current in the photodetector.
Photo_SignalP    = 1e-6
Power_interval   = np.linspace(0,1000,100000) # In [w]. Power interval for the photodetector domain in photodetector SNR plot. 

#unc_func         = uphc.UQ_Photodetector # Function describing Photodetector uncertainty. Further informaion in "UQ_Photonics_Classes.py" comments.

# Responsivity [A/W]
R = Photo_efficiency*e*Wavelength/(h*c)


#### Thermal noise

In [ ]:
# SNR 
SNR_thermal_w      = [((R**2)/(4*k*Temperature*Photo_BandWidth/Load_Resistor))*(Power_interval/1000)**2] # SNR  watts
SNR_thermal     = [10*np.log10(SNR_thermal_w)][0]# SNR  dB
# Noise
Thermal_noise_w=4*k*Temperature*Photo_BandWidth/Load_Resistor
Thermal_noise_dB=10*np.log10(Thermal_noise_w)

print('Thermal_noise[w] = '+ str(Thermal_noise_w))
print('Thermal_noise[dB] = '+ str(Thermal_noise_dB))

#### Shot noise

In [ ]:
#SNR
SNR_shot_w  = [((R**2)/(2*e*R*Photo_BandWidth))*(Power_interval)/1000]
SNR_shot    = [10*np.log10(SNR_shot_w )][0]

#Noise
Shot_noise_w = 2*e*Dark_Current*Photo_BandWidth
Shot_noise_dB = 10*np.log10(Shot_noise_w)

print('Shot_noise[w] = '+ str(Shot_noise_w))
print('Shot_noise[dB] = '+ str(Shot_noise_dB))

#### Dark Current noise

In [ ]:
#SNR
SNR_DarkCurrent_w  = [((R**2)/(2*e*Dark_Current*Photo_BandWidth))*(Power_interval/1000)**2]
SNR_DarkCurrent = [10*np.log10(SNR_DarkCurrent_w)][0]

#Noise
Dark_current_noise_w = 2*e*Dark_Current*Photo_BandWidth
Dark_current_noise_dB = 10*np.log10(Dark_current_noise_w)

print('DarkCurrent_Noise[w] = '+ str(Dark_current_noise_w))
print('DarkCurrent_Noise[dB] = '+ str(Dark_current_noise_dB))

### Combining uncertainties in the photodetector:
To combine this uncertainties we use GUM

In [ ]:
Photodetector_comb_w  = np.sqrt((Thermal_noise_w)**2+(Shot_noise_w)**2+(Dark_current_noise_w)**2)
Photodetector_comb_dB = 10*np.log10(Photodetector_comb_w)  #[dB]

print('Photodetector_comb[w] = '+ str(Photodetector_comb_w))
print('Photodetector_comb[dB] = '+ str(Photodetector_comb_dB))

### **Optical amplifier**
Optical amplifier noise is dominated and, therefore characterized by the amplified spontaneous emission (ASE)

Input parameters:

In [ ]:
FigureNoise = 1 # Degradation of the SNR in dB. Given by the manufature
OA_Gain     = 30 #dB

Optical amplifier noise estimation:

In [ ]:
Optical_amplifier_noise_w  = np.array([((10**(FigureNoise/10))*h*(c/Wavelength)*10**(OA_Gain/10))]) # ASE noise --> (**Optics and Photonics) Bishnu P. Pal - Guided Wave Optical Components and Devices_ Basics, Technology, and Applications -Academic Press (2005)
UQ_Optical_amplifier_noise_dB = np.array([10*np.log10((10**(FigureNoise/10))*h*(c/Wavelength)*10**(OA_Gain/10))]) # ASE noise --> (**Optics and Photonics) Bishnu P. Pal - Guided Wave Optical Components and Devices_ Basics, Technology, and Applications -Academic Press (2005)

print('Optical_amplifier_noise[w] = '+ str(Optical_amplifier_noise_w[0]))
print('Optical_amplifier_noise[dB] = '+ str(UQ_Optical_amplifier_noise_dB[0]))

### **Expanding uncertainties**
Gum is applied to expand uncertainties and get global lidar uncertainty. The model combines noise from the different lidar modules and computes expanded uncertainty:

In [ ]:
Photonics_comb_w=np.sqrt((Photodetector_comb_w)**2+(Optical_amplifier_noise_w)**2)
Photonics_comb_dB=10*np.log10(Photonics_comb_w)

print('Photonics_comb[w] = '  + str(Photonics_comb_w[0]))
print('Photonics_comb[dB] = ' + str(Photonics_comb_dB[0]))

### Plotting photodetector SNR

In [ ]:
plt.figure(figsize=(10,6)  )
plt.grid(axis='both')
plt.plot(10*np.log10(Power_interval),SNR_thermal[0],label='Thermal')
plt.plot(10*np.log10(Power_interval),SNR_shot[0],label='Shot')
plt.plot(10*np.log10(Power_interval),SNR_DarkCurrent[0],label='Dark current')
plt.xlabel('Input Signal optical power [dBm]',fontsize=27)#,orientation=plot_param['tick_labelrotation'])
plt.ylabel('SNR [dB]',fontsize=27)#,orientation=plot_param['tick_labelrotation'])
plt.title('Lidar pointing accuracy (VAD)',fontsize=29)
plt.legend()

## Hosting Qlunc
Provide a place for Qlunc's documentation

![Sphinx4Qlunc](./Tutorial_Images/SphinxQlunc.PNG)